# Submit LLM 34B Model in 5 hours!
This notebook demonstrates how to submit a LLM 34B model in only 5 hours! Amazing! The key tricks are:
* use vLLM (for speed)
* use AWQ 4bit quantization (to avoid GPU VRAM OOM)
* limit input size to 1024 tokens (for speed)
* limit output size to 1 token (for speed)

# Pip Install vLLM
The package vLLM is an incredibly fast LLM inference library! The vLLM that is installed in Kaggle notebooks will produce errors, therefore we need to reinstall vLLM. The code below was taken from notebook [here][1]

[1]: https://www.kaggle.com/code/lewtun/numina-1st-place-solution

In [ ]:
import os, math, numpy as np
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [ ]:
%%time
!pip uninstall -y torch
!pip install -U --no-index --find-links=/kaggle/input/vllm-whl -U vllm
!pip install -U --upgrade /kaggle/input/vllm-t4-fix/grpcio-1.62.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -U --upgrade /kaggle/input/vllm-t4-fix/ray-2.11.0-cp310-cp310-manylinux2014_x86_64.whl

# Load 34B Quantized Model with vLLM!
We will load and use LLM 34B Bagel [here][1]. This is a strong model.

[1]: https://huggingface.co/jondurbin/bagel-34b-v0.2

In [ ]:
import vllm

llm = vllm.LLM(
    "/kaggle/input/bagel-v3-343",
    quantization="awq",
    tensor_parallel_size=2, 
    gpu_memory_utilization=0.95, 
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
    max_model_len=1024,
    #distributed_executor_backend="ray",
)
tokenizer = llm.get_tokenizer()

# Load Test Data
During **commit** we load 128 rows of train to compute CV score. During **submit**, we load the test data.

In [ ]:
import pandas as pd
VALIDATE = 128

test = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/test.csv") 
if len(test)==3:
    test = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")
    test = test.iloc[:VALIDATE]
print( test.shape )
test.head(1)

# Engineer Prompt
If we want to submit zero shot LLM, we need to experiment with different system prompts to improve CV score. If we finetune the model, then system is not as important because the model will learn from the targets what to do regardless of which system prompt we use.

We use a logits processor to force the model to output the 3 tokens we are interested in.

In [ ]:
from typing import Any, Dict, List
from transformers import LogitsProcessor
import torch

choices = ["A","B","tie"]

KEEP = []
for x in choices:
    c = tokenizer.encode(x,add_special_tokens=False)[0]
    KEEP.append(c)
print(f"Force predictions to be tokens {KEEP} which are {choices}.")

class DigitLogitsProcessor(LogitsProcessor):
    def __init__(self, tokenizer):
        self.allowed_ids = KEEP
        
    def __call__(self, input_ids: List[int], scores: torch.Tensor) -> torch.Tensor:
        scores[self.allowed_ids] += 100
        return scores

In [ ]:
sys_prompt = """Please read the following prompt and two responses. Determine which response is better.
If the responses are relatively the same, respond with 'tie'. Otherwise respond with 'A' or 'B' to indicate which is better."""

In [ ]:
SS = "#"*25 + "\n"

In [ ]:
all_prompts = []
for index,row in test.iterrows():
    
    a = " ".join(eval(row.prompt, {"null": ""}))
    b = " ".join(eval(row.response_a, {"null": ""}))
    c = " ".join(eval(row.response_b, {"null": ""}))
    
    prompt = f"{SS}PROMPT: "+a+f"\n\n{SS}RESPONSE A: "+b+f"\n\n{SS}RESPONSE B: "+c+"\n\n"
    
    formatted_sample = sys_prompt + "\n\n" + prompt
    
    all_prompts.append( formatted_sample )

# Infer Test
We infer test using fast vLLM. We ask vLLM to output probabilties of the top 5 tokens considered to be predicted in the first token. We also limit prediction to 1 token to increase inference speed.

Based on the speed it takes to infer 128 train samples, we can deduce how long inferring 25,000 test samples will take.

In [ ]:
%%time

from time import time
start = time()

logits_processors = [DigitLogitsProcessor(tokenizer)]
responses = llm.generate(
    all_prompts,
    vllm.SamplingParams(
        n=1,  # Number of output sequences to return for each prompt.
        top_p=0.9,  # Float that controls the cumulative probability of the top tokens to consider.
        temperature=0,  # randomness of the sampling
        seed=777, # Seed for reprodicibility
        skip_special_tokens=True,  # Whether to skip special tokens in the output.
        max_tokens=1,  # Maximum number of tokens to generate per output sequence.
        logits_processors=logits_processors,
        logprobs = 5
    ),
    use_tqdm = True
)

end = time()
elapsed = (end-start)/60. #minutes
print(f"Inference of {VALIDATE} samples took {elapsed} minutes!")

In [ ]:
submit = 25_000 / 128 * elapsed / 60
print(f"Submit will take {submit} hours")

# Extract Inference Probabilites
We now extract the probabilties of "A", "B", "tie" from the vLLM predictions.

In [ ]:
results = []
errors = 0

for i,response in enumerate(responses):
    try:
        x = response.outputs[0].logprobs[0]
        logprobs = []
        for k in KEEP:
            if k in x:
                logprobs.append( math.exp(x[k].logprob) )
            else:
                logprobs.append( 0 )
                print(f"bad logits {i}")
        logprobs = np.array( logprobs )
        logprobs /= logprobs.sum()
        results.append( logprobs )
    except:
        #print(f"error {i}")
        results.append( np.array([1/3., 1/3., 1/3.]) )
        errors += 1
        
print(f"There were {errors} inference errors out of {i+1} inferences")
results = np.vstack(results)

# Create Submission CSV

In [ ]:
sub = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/sample_submission.csv")

if len(test)!=VALIDATE:
    sub[["winner_model_a","winner_model_b","winner_tie"]] = results
    
sub.to_csv("submission.csv",index=False)
sub.head()

# Compute CV Score

In [ ]:
if len(test)==VALIDATE:
    true = test[['winner_model_a','winner_model_b','winner_tie']].values
    print(true.shape)

In [ ]:
if len(test)==VALIDATE:
    from sklearn.metrics import log_loss
    print(f"CV loglosss is {log_loss(true,results)}" )